In [1]:
import pandas as pd
import sklearn
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm , preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

/home/administrator/.local/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


## Function to calculate the scores

In [2]:
def mean_score(scoring):
    return {i:j.mean() for i,j in scoring.items()}

## Loading the data
- Load the data and clean it for unique values, later shuffle it with specific seeding

In [3]:
# Get the parent directory of the current folder
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Specify the file path for dataset.csv in the parent folder
filename = os.path.join(parent_dir, "dataset.csv")
df = pd.read_csv(filename)

## Data Preprocessing

In [4]:
df.columns 

Index(['frequency_of_a_tags', 'frequency_of_alltags', 'presence_of_iframes',
       'presence_of_popups', 'right_click_disabling', 'num_of_redirects',
       'checking_cookies', 'checking_sfh', 'request_url', 'url-of-anchor',
       'links_in_meta_img', 'Result'],
      dtype='object')

In [5]:
data_no_duplicates = df.drop_duplicates()
df = data_no_duplicates.dropna()

In [6]:
# Investigate unique values in the each column
unique_frequency_a = df['frequency_of_a_tags'].unique()
unique_all = df['frequency_of_alltags'].unique()
unique_iframes = df['presence_of_iframes'].unique()
unique_popups = df['presence_of_popups'].unique()
unique_rightclick = df['right_click_disabling'].unique()
unique_redirects = df['num_of_redirects'].unique()
unique_cookies = df['checking_cookies'].unique()
unique_sfh = df['checking_sfh'].unique()
unique_request = df['request_url'].unique()
unique_url = df['url-of-anchor'].unique()
unique_meta = df['links_in_meta_img'].unique()
unique_R = df['Result'].unique()

print(unique_frequency_a)
print(unique_all)
print(unique_iframes)
print(unique_popups)
print(unique_rightclick)
print(unique_redirects)
print(unique_cookies)
print(unique_sfh)
print(unique_request)
print(unique_url)
print(unique_meta)
print(unique_R)

['-1' '1' 'frequency_of_a_tags']
['-1' '1' 'frequency_of_alltags']
['1' 'presence_of_iframes']
['1' 'presence_of_popups']
['1' 'right_click_disabling']
['0' '1' '-99' '-1' 'num_of_redirects']
['1' '-1' 'checking_cookies' '-99']
['-1' 'checking_sfh']
['-1' '1' '0' 'request_url']
['-1' '0' '1' 'url-of-anchor']
['-1' '1' '0' 'links_in_meta_img']
['1' 'Result' '-1']


### Remove all the values other than {-1, 0, 1} from the data

In [7]:
validValues = {-1, 0, 1}

for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')
    df = df[df[column].isin(validValues)]

# Convert all columns to integers (again after the filtering)
df = df.astype(int)

/tmp/ipykernel_35029/837780899.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_numeric(df[column], errors='coerce')


In [8]:
# Now you can use the DataFrame with valid integer values (1, -1, and 0)
print(df)

      frequency_of_a_tags  frequency_of_alltags  presence_of_iframes  \
0                      -1                    -1                    1   
2                       1                     1                    1   
7                      -1                    -1                    1   
12                      1                     1                    1   
15                      1                     1                    1   
25                      1                     1                    1   
28                     -1                    -1                    1   
36                     -1                    -1                    1   
56                     -1                    -1                    1   
60                      1                     1                    1   
62                     -1                     1                    1   
66                     -1                     1                    1   
86                      1                    -1                 

In [9]:
df.shape

(53, 12)

### Data Preprocessing
- Shuffle the data, and split it into train and test data

In [10]:
df = sklearn.utils.shuffle(df)
X = df.drop("Result",axis=1).values
X = preprocessing.scale(X)
y = df['Result'].values
df.head()

,frequency_of_a_tags,frequency_of_alltags,presence_of_iframes,presence_of_popups,right_click_disabling,num_of_redirects,checking_cookies,checking_sfh,request_url,url-of-anchor,links_in_meta_img,Result
366,-1,-1,1,1,1,1,-1,-1,1,-1,0,1
1104,-1,-1,1,1,1,0,-1,-1,-1,-1,1,-1
200,-1,-1,1,1,1,1,-1,-1,1,0,-1,1
100,-1,-1,1,1,1,1,-1,-1,1,0,1,1
552,-1,-1,1,1,1,0,1,-1,1,-1,1,1


## Evalution Metrics

In [11]:
scoring = {'accuracy': 'accuracy',
           'recall': 'recall',
           'precision': 'precision',
           'f1': 'f1'}
fold_count=10

## Random Forest

In [12]:
rforest_clf=RandomForestClassifier()
cross_val_scores = cross_validate(rforest_clf, X, y, cv=fold_count, scoring=scoring)
rforest_clf_score = mean_score(cross_val_scores)

print(f"fit time = {rforest_clf_score['fit_time']}")
print(f"score time = {rforest_clf_score['score_time']}")
print(f"accuracy = {rforest_clf_score['test_accuracy']}")
print(f"recall = {rforest_clf_score['test_recall']}")
print(f"precision = {rforest_clf_score['test_precision']}")
print(f"f1 = {rforest_clf_score['test_f1']}")

/home/administrator/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=10.
  warnings.warn(


fit time = 0.6307787418365478
score time = 0.05594546794891357
accuracy = 0.8733333333333333
recall = 0.9800000000000001
precision = 0.8866666666666667
f1 = 0.9284848484848485
